In [1]:
%ls

Datasets/                                 nltk_data/
GermanT5-RP-Mod/                          ray_results/
RP-Mod/                                   results/
RP-Mod-GermanT5-oscar-german-small-el32/  t5_base_imdb_sentiment/
T5FineTuner.py                            t5_german_small_rp_mod/
aclImdb/                                  t5_german_small_rp_mod_2/
aclImdb_v1.tar.gz                         t5_german_small_rp_mod_3/
arguments_test_dir/                       t5_imdb_sentiment/
create_t5_embeddings.ipynb                wandb/
lightning_logs/


In [30]:
from T5FineTuner import T5FineTuner, RPDataset
from utils import get_folds
import torch
import argparse
from transformers import T5Tokenizer
from torch.utils.data import Dataset, DataLoader

DATASET = "RP-Crowd-3"
MODEL_NAME_OR_PATH = "GermanT5/t5-efficient-oscar-german-small-el32"
WANDB_PROJECT_NAME = "rp-crowd-3-folds-t5-efficient-small-el32"
OUTPUT_DIR = "./GermanT5-RP-Mod/t5-efficient-oscar-german-small-el32/"
SOURCE = f"./Datasets/{DATASET}-folds.csv"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME_OR_PATH)
checkpoint_path = "./GermanT5-RP-Mod/t5-efficient-oscar-german-small-el32/lr-0.0004-wd-0.1/epoch=3-val_accuracy=0.74-val_loss=0.28.ckpt"
train_inputs, train_targets, val_inputs, val_targets = get_folds(SOURCE)

train_dataset = RPDataset(tokenizer, train_inputs, train_targets)
valid_dataset = RPDataset(tokenizer, val_inputs, val_targets)

checkpoint = torch.load(checkpoint_path)
hparams = checkpoint["hyper_parameters"]
hparams["train_dataset"] = train_dataset
hparams["val_dataset"] = valid_dataset
new_args = argparse.Namespace(**hparams)

new_model = T5FineTuner.load_from_checkpoint(checkpoint_path, hparams=new_args)


### Load checkpoint of Encoder Only Model

In [32]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sun Jul 24 12:56:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:D8:00.0 Off |                  Off |
| 35

In [34]:
from classification_classes import Enc1T5, RPClassificationDataset, load_dataset
import torch
import argparse
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader

DATASET = "RP-Crowd-2"
MODEL_NAME_OR_PATH = "GermanT5/t5-efficient-oscar-german-small-el32"
source = f"./Datasets/{DATASET}-folds.csv"

# tokenizer = AutoTokenizer.from_pretrained()
# train_dataset, valid_dataset = load_dataset(source)

checkpoint_path = "/home/dobby/bestmodels/enc1t5/epoch=3-val_accuracy=0.85.ckpt"                 
checkpoint = torch.load(checkpoint_path)
hparams = checkpoint["hyper_parameters"]
# hparams["train_dataset"] = train_dataset
# hparams["val_dataset"] = valid_dataset
new_args = argparse.Namespace(**hparams)

# now I am confused... what do I do here??
model = Enc1T5.load_from_checkpoint(checkpoint_path, hparams=new_args)

Some weights of the model checkpoint at GermanT5/t5-efficient-oscar-german-small-el32 were not used when initializing T5EncoderModel: ['decoder.block.1.layer.0.SelfAttention.q.weight', 'decoder.block.4.layer.0.SelfAttention.v.weight', 'decoder.block.4.layer.0.SelfAttention.k.weight', 'decoder.block.3.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.2.layer.1.EncDecAttention.k.weight', 'decoder.block.5.layer.2.layer_norm.weight', 'decoder.block.1.layer.0.layer_norm.weight', 'decoder.block.4.layer.1.layer_norm.weight', 'decoder.block.3.layer.2.layer_norm.weight', 'decoder.block.5.layer.0.SelfAttention.v.weight', 'decoder.block.4.layer.2.DenseReluDense.wi.weight', 'decoder.embed_tokens.weight', 'decoder.block.3.layer.1.layer_norm.weight', 'decoder.block.1.layer.1.EncDecAttention.k.weight', 'decoder.block.2.layer.0.SelfAttention.v.weight', 'decoder.block.5.layer.1.EncDecAttention.o.weight', 'decoder.block.2.layer.0.SelfAttention.o.weight',

### Get False Positives

In [ ]:
# apply the model to the entire validation dataset


In [35]:
import numpy as np
import scipy as sp
import shap

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)

# define a score function
def f(x):
    """A function which takes a list of examples, x and returns a list of scores, val"""
    # print(len(x))
    # print(x)
    batch = tokenizer.batch_encode_plus(
        list(x), max_length=512, padding=True, truncation=True, return_tensors="pt"
    )
    outputs = model(batch["input_ids"], batch["attention_mask"])["prediction"].detach().numpy()
    # print(tv)
    # print(outputs)
    # apply softmax to each row??
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    # print(scores)
    val = scores[:,1] # use one vs rest logit units
    # print(val)
    # val needs to be a list of the score
    return val

false_pos = np.load("./false_pos/RP-Mod-false-pos.csv.npy", mmap_mode="r")
false_pos = [s[15:] for s in false_pos]
# f(list(false_pos[:10]))

explainer = shap.Explainer(f, tokenizer)


# shap_values = explainer(false_pos[:10], batch_size=8)
shap_values = explainer(false_pos, fixed_context=1, batch_size=8)

Partition explainer:   5%|▌         | 7/139 [00:16<05:55,  2.69s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer:   9%|▉         | 13/139 [00:37<04:19,  2.06s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer:  12%|█▏        | 16/139 [00:55<08:06,  3.96s/it]

...

Partition explainer:  97%|█████████▋| 135/139 [12:53<00:42, 10.74s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 100%|██████████| 139/139 [13:07<00:00,  5.04s/it]

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 140it [13:13,  5.88s/it]                         


In [41]:
#TODO: what is the correlation between SHAP score and how underrepresented these words are 
# in the dataset?
import pandas as pd
import scipy as sp

mod_folds_df = pd.read_csv("./Datasets/RP-Mod-folds.csv")
train_df = mod_folds_df.loc[mod_folds_df["train_test_split"] == 0]

def score_underrepresentation(df, word, label_attr="label", text_attr="text"):
    occ = df[np.array(df[text_attr].str.contains(word), dtype=bool)]
    num_pos = len(occ.loc[occ[label_attr] == 1].drop_duplicates())
    num_neg = len(occ.loc[occ[label_attr] == 0].drop_duplicates())
    return num_pos/(num_neg + 1)


In [ ]:
#TODO: create dataframe, where the columns are words, underrepresentation score, SHAP score


In [8]:
shap_values[0]

.values =
array([-0.00050354, -0.05605525, -0.00404001,  0.09449696,  0.0202925 ,
       -0.04756196, -0.00803574,  0.04582678, -0.0513271 ,  0.14837754,
       -0.01956507, -0.00458249])

.base_values =
0.6096603870391846

.data =
array(['Sehr', ' sympathisch', ' diese', ' Frau', ' und', ' das', ' sagt',
       ' hier', ' ein ', ' Mann', ' ', ''], dtype='<U12')

In [54]:
import heapq
from collections import OrderedDict

words = OrderedDict()

for val in shap_values:
    for i in range(len(val.values)):
        score = val.values[i]
        word = val.data[i]
        if word not in words.keys():
            words[word] = [score]
        else:
            words[word].append(score)
for key in words.keys():
    words[key] = np.mean(words[key])
words
# largest = heapq.nlargest(100, words, key=words.get)
largest = words.keys()
# for word in largest:
#     print(word, words[word])

In [55]:
underrepresentation_scores = []
shap_scores = []
for word in words.keys():
    if word in largest:
        underrepresentation_scores.append(score_underrepresentation(train_df, word))
        shap_scores.append(words[word])
sp.stats.pearsonr(underrepresentation_scores, shap_scores)

(0.1416182821691077, 4.3566645025812035e-11)

In [56]:
df = pd.DataFrame({"words": words.keys(), "shap": shap_scores, "underrepresented": underrepresentation_scores})
df

,words,shap,underrepresented
0,Sehr,-0.000504,0.583333
1,sympathisch,-0.056055,1.000000
2,diese,0.020005,0.880952
3,Frau,0.086792,0.987342
4,und,-0.004066,0.772222
...,...,...,...
2142,wählen,0.002103,0.711538
2143,richtig,0.035320,0.594406
2144,Arg,-0.003833,0.461538
2145,iten,0.017260,0.721088


In [58]:
df.to_csv("./false_pos/shap_underrepresented_scores.csv")

In [29]:
n_largest = 50
largest = heapq.nlargest(n_largest, words, key=words.get)

underrepresentation_scores = []
shap_scores = []
for word in words.keys():
    if word in largest:
        underrepresentation_scores.append(score_underrepresentation(train_df, word))
        shap_scores.append(words[word])

sp.stats.pearsonr(underrepresentation_scores, shap_scores)

(-0.3067348171046196, 0.03026463078329042)

In [18]:
list(words.keys())[np.argmin(underrepresentation_scores)]

'brüh'

In [71]:
shap.plots.text(shap_values[0])

In [32]:
#TODO: remove "Kurden" (or other words which SHAP identifies as import) 
# from the comment and then see what score the model gives it :) 
import numpy as np

txt = "Ich stelle mal folgende Vermutung an so wie es auch bei Tätern hier immer dargestellt wird Einzeltäter verwirrt traumatisiert. Liebe über alles"
def score_text(txt):

    input_ids = torch.tensor([tokenizer.encode(txt)])
    pred = model(input_ids)["prediction"].detach().numpy()
    # print(pred)
    scores = (np.exp(pred).T / np.exp(pred).sum(-1)).T
    return scores
score_text(txt)

array([[0.50282335, 0.49717662]], dtype=float32)

In [54]:
score_text("Ich liebe die Umwelt und mein Leben ist sehr schoen Diese Artikel wurde sehr gut geschrieben danke Mann" + "Liebe alles" * 100)

array([[0.5951737 , 0.40482628]], dtype=float32)

In [ ]:
score_text("ich liebe dich")

In [58]:
score_text("Ich liebe Katzen "*100)
# hello??? this can't be right :(

array([[0.6666064, 0.3333936]], dtype=float32)

In [59]:
true_negatives = np.load("./true_neg/RP-Mod-true-neg.npy")
sum([score_text(true_negatives[i])[0][1] >= 0.5 for i in range(len(true_negatives))])/len(true_negatives)
# score_text(true_negatives[6])

0.3873578

In [63]:
true_negatives[7]

'classification: Weil das Land damit einen einstelligen Milliardenbetrag an Subventionen spart '

In [24]:
f(false_pos[1:2])

array([0.22180541], dtype=float32)

In [23]:
false_pos[:10]

memmap(['classification Sehr sympathisch diese Frau und das sagt hier ein  Mann ',
        'classification Kein Artikel wert',
        'classification Na ja Wenigstens hat sie diesmal nicht Deutschland verbrüht Schonmal eine Entwicklungsstufe weiter',
        'classification Hallo nehmt doch die Rocker aus Erkrath Unterfeldhaus Die Jungens sind froh über eine solche Arbeit Wer sich nicht benimmt der fliegt und zwar raus',
        'classification Wildtiere haben im Zirkus nichts verloren   Inkompetente haben in der Politik nichts zu suchen ',
        'classification Ich stelle mal folgende Vermutung an so wie es auch bei Tätern mit Migrationshintergrund hier immer dargestellt wirdEinzeltäter verwirrt traumatisiert hat nix mit den Rechtsextremen zu tun',
        'classification Ich freue mich zu wissen das Deutschland in Brüssel von einer vertreten wird die Islamwissenschaft und Turkologie studiert hat  Sehr hilfreich für die vielen neuen Gäste Ska wird ganz ganz viel Verständnis für die

In [15]:
false_pos[:10]

memmap(['classification Sehr sympathisch diese Frau und das sagt hier ein  Mann ',
        'classification Kein Artikel wert',
        'classification Na ja Wenigstens hat sie diesmal nicht Deutschland verbrüht Schonmal eine Entwicklungsstufe weiter',
        'classification Hallo nehmt doch die Rocker aus Erkrath Unterfeldhaus Die Jungens sind froh über eine solche Arbeit Wer sich nicht benimmt der fliegt und zwar raus',
        'classification Wildtiere haben im Zirkus nichts verloren   Inkompetente haben in der Politik nichts zu suchen ',
        'classification Ich stelle mal folgende Vermutung an so wie es auch bei Tätern mit Migrationshintergrund hier immer dargestellt wirdEinzeltäter verwirrt traumatisiert hat nix mit den Rechtsextremen zu tun',
        'classification Ich freue mich zu wissen das Deutschland in Brüssel von einer vertreten wird die Islamwissenschaft und Turkologie studiert hat  Sehr hilfreich für die vielen neuen Gäste Ska wird ganz ganz viel Verständnis für die

In [25]:
from sklearn import metrics
val_dataloader = DataLoader(valid_dataset, batch_size=64)

# for batch in val_dataloader:
batch = next(iter(val_dataloader))
labels = batch["target_ids"]
outs = new_model.model.generate(input_ids=batch["source_ids"], 
                            attention_mask=batch["source_mask"], 
                            max_length=2)

dec = [tokenizer.decode(ids) for ids in outs]
target = [tokenizer.decode(label) for label in labels]

new_outputs = [s[6:] for s in dec]
new_targets = [s[:-4] for s in target]

f1 = metrics.f1_score(new_targets, new_outputs, pos_label="problematisch")
print(f1)

0.7419354838709677
